In [ ]:
pip install wfdb

In [ ]:
import wfdb
from copy import deepcopy
import pywt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
import os
import csv
from keras import backend as K
from keras.models import load_model
from scipy.signal import find_peaks
from scipy.signal import butter, filtfilt

In [ ]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, UpSampling1D, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MeanSquaredError

# Load the CSV file into a DataFrame
data = pd.read_csv('/content/drive/MyDrive/ECG/incart_DB/Incart_csv_3/I01.csv')

# Extract the Lead 2 ECG signal
ecg_signal = data['II'].values

# Define the sampling frequency
fs = 360  # Assuming the data is resampled at 360 Hz

# Define the cutoff frequencies for the bandpass filter
lowcut = 1  # Lower cutoff frequency in Hz
highcut = 30  # Upper cutoff frequency in Hz
nyquist = 0.5 * fs
low = lowcut / nyquist
high = highcut / nyquist
order = 4  # Filter order (adjust as needed)

# Apply the bandpass filter to the ECG signal
filtered_ecg_signal = signal.lfilter(*signal.butter(order, [low, high], btype='band'), ecg_signal)

# Divide the filtered ECG signal into non-overlapping ten-second segments
segment_length = 10 * fs  # Ten seconds of data
segments = [filtered_ecg_signal[i:i+segment_length] for i in range(0, len(filtered_ecg_signal), segment_length)]

# Scale the segments to the [-1, 1] range
scaled_segments = [(segment - np.mean(segment)) / np.max(np.abs(segment - np.mean(segment))) for segment in segments]

# Convert the scaled segments into a data matrix X
X = np.array(scaled_segments)

# Split the data into training and validation sets
X_train, X_val = train_test_split(X, test_size=0.2, random_state=32)

# Build the 1D-CAE model for initial beat segmentation
model = Sequential()
model.add(Conv1D(filters=32, kernel_size=3, activation='relu', padding='same', input_shape=(segment_length, 1)))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=16, kernel_size=3, activation='relu', padding='same'))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=16, kernel_size=3, activation='relu', padding='same'))
model.add(UpSampling1D(size=2))
model.add(Conv1D(filters=32, kernel_size=3, activation='relu', padding='same'))
model.add(UpSampling1D(size=2))
model.add(Conv1D(filters=1, kernel_size=3, activation='tanh', padding='same'))

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss=MeanSquaredError())

# Train the model
model.fit(X_train, X_train, validation_data=(X_val, X_val), epochs=10, batch_size=32)

# Apply the beat model to the filtered ECG signal
beat_detected_signal = model.predict(X)


In [ ]:
len(segments)

In [ ]:
# Plot the original ECG signal
plt.figure(figsize=(10, 4))
plt.plot(ecg_signal, color='blue')
plt.title('Original ECG Signal')
plt.xlabel('Sample')
plt.ylabel('Amplitude')
plt.grid(True)
plt.show()

# Plot the beat-detected signal
plt.figure(figsize=(10, 4))
plt.plot(filtered_ecg_signal, color='red')
plt.title('De-noised Signal')
plt.xlabel('Sample')
plt.ylabel('Amplitude')
plt.grid(True)
plt.show()


In [ ]:
original_segment1 = filtered_ecg_signal[0]
extracted_segment1 = beat_detected_signal[0]

# Plot the original ECG signal
plt.figure(figsize=(10, 4))
plt.plot(filtered_ecg_signal, color='blue')
plt.title('Original ECG Signal')
plt.xlabel('Sample')
plt.ylabel('Amplitude')
plt.grid(True)
plt.show()

# Plot the beat-detected signal
plt.figure(figsize=(10, 4))
plt.plot(beat_detected_signal[0], color='red')
plt.title('Beat-Detected Signal')
plt.xlabel('Sample')
plt.ylabel('Amplitude')
plt.grid(True)
plt.show()


In [ ]:
beat_detected_signal.shape

In [ ]:
X.shape

In [ ]:
# Initialization
Fs = 257  # Sampling frequency of the denoised ECG signal

pk_thr = 0.25  # R-peak threshold
rr_thr = int(0.25 * Fs)  # RR-interval threshold (in samples)
missed_thr = 0  # Minimum separation threshold between consecutive peaks for identifying missed beats

temp_locs = np.array([], dtype=int)
pk_vec = np.array([], dtype=float)
ppi_vec = np.array([], dtype=int)

for beat in beat_detected_signal:
    # Step 1: Identify peak locations with minimum amplitude of pk_thr units and separated by at least rr_thr samples
    peaks_pos, _ = find_peaks(beat.flatten(), height=pk_thr, distance=rr_thr)
    peaks_neg, _ = find_peaks(-beat.flatten(), height=pk_thr, distance=rr_thr)

    # Combine positive and negative peaks
    peaks = np.concatenate((peaks_pos, peaks_neg))

    # Step 2: Update temp_locs with the locations of the found peaks
    temp_locs = np.concatenate((temp_locs, peaks))

    # Step 3: Update pk_vec with the amplitudes of these peaks
    peak_amplitudes = beat.flatten()[peaks]
    pk_vec = np.concatenate((pk_vec, peak_amplitudes))

    # Step 4: Update ppi_vec with peak-to-peak interval (PPI) values computed from these peaks
    peak_intervals = np.diff(peaks)
    ppi_vec = np.concatenate((ppi_vec, peak_intervals))

# Convert temp_locs, pk_vec, and ppi_vec to numpy arrays for further analysis if needed
temp_locs = np.array(temp_locs)
pk_vec = np.array(pk_vec)
ppi_vec = np.array(ppi_vec)


In [ ]:
beat.shape

In [ ]:
ppi_vec.shape

In [ ]:
pk_vec.shape

In [ ]:
temp_locs.shape

In [ ]:
peaks_pos.shape

In [ ]:
peaks_neg.shape

In [ ]:
peaks.shape

In [ ]:
for segment_index, segment in enumerate(beat_detected_signal):
    # Extract the denoised ECG signal
    ecg_signal = segment[:, 0]

    # Plot the segment with detected peaks
    plt.figure(figsize=(10, 4))
    plt.plot(ecg_signal)
    plt.plot(peaks, ecg_signal[peaks], 'ro', markersize=4)
    plt.xlabel('Time')
    plt.ylabel('Denoised ECG Signal')
    plt.title(f'Reconstructed Segment {segment_index+1} with Initial Peak Detection')
    plt.show()

In [ ]:
# Step 2: Missed Beat Detection
missed_thr = np.zeros(len(ppi_vec), dtype=int)  # Initialize missed_thr vector
missed_peaks = np.array([], dtype=int)  # Initialize missed peaks

for i in range(len(ppi_vec)):
    if i == 0:
        missed_thr[i] = int(1.5 * Fs)
    else:
        missed_thr[i] = int(1.5 * ppi_vec[i-1])

    if i < len(beat_detected_signal):
        # Find peaks within the interval with a minimum amplitude of 0.05 units
        peaks, _ = find_peaks(beat_detected_signal[i].flatten(), height=0.05)

        # Update temp_locs, pk_vec, and ppi_vec vectors with the new peaks
        temp_locs = np.concatenate((temp_locs, peaks))
        pk_vec = np.concatenate((pk_vec, beat_detected_signal[i].flatten()[peaks]))
        ppi_vec = np.concatenate((ppi_vec, np.diff(peaks)))

        # Update missed_peaks with the new peaks found
        missed_peaks = np.concatenate((missed_peaks, peaks))

# Convert missed_peaks to numpy array if needed
missed_peaks = np.array(missed_peaks)


In [ ]:
missed_peaks.shape

In [ ]:
for segment_index, segment in enumerate(beat_detected_signal):
    # Extract the denoised ECG signal
    ecg_signal = segment[:, 0]
    # missed_locs = temp_locs[missed_peaks]
    # Plot the segment with detected peaks
    plt.figure(figsize=(10, 4))
    plt.plot(ecg_signal)
    plt.plot(missed_peaks, ecg_signal[missed_peaks], 'ro', markersize=4)
    plt.xlabel('Time')
    plt.ylabel('Beat Extracted ECG Signal')
    plt.title(f'Reconstructed Segment {segment_index+1} with Missed Peak Detection')
    plt.show()

In [ ]:
# Step 3: False Beat Removal
falsebeat_thr = np.zeros(len(ppi_vec), dtype=int)  # Initialize falsebeat_thr vector

for i in range(len(ppi_vec)):
    if i == 0:
        falsebeat_thr[i] = int(0.33 * Fs)
    else:
        falsebeat_thr[i] = int(0.35 * ppi_vec[i-1])

# Obtain indices and amplitudes of peaks within intervals smaller than falsebeat_thr
indices = []
amplitudes = []

for interval in ppi_vec:
    if interval < falsebeat_thr[i]:
        # Find the indices and amplitudes of peaks within the interval
        start_idx = temp_locs[i-1]
        end_idx = temp_locs[i]
        interval_indices = np.arange(start_idx, end_idx+1)
        interval_amplitudes = pk_vec[start_idx:end_idx+1]

        # Retain the index with the higher peak amplitude and delete the other one
        if len(interval_indices) > 1:
            max_amplitude_idx = np.argmax(interval_amplitudes)
            indices.append(interval_indices[max_amplitude_idx])
            amplitudes.append(interval_amplitudes[max_amplitude_idx])
        else:
            indices.append(interval_indices[0])
            amplitudes.append(interval_amplitudes[0])

# Update temp_locs, pk_vec, and ppi_vec vectors accordingly
temp_locs = np.array(indices)
pk_vec = np.array(amplitudes)
ppi_vec = np.diff(temp_locs)


In [ ]:
falsebeat_thr

In [ ]:
temp_locs

In [ ]:
ppi_vec

In [ ]:
# Step4: Threshold update
rr_thr = max((0.25 * Fs), 0.5 * rr_thr + 0.5 * np.median(ppi_vec))
pk_thr = 0.5 * np.median(pk_vec)


In [ ]:
# Step 5: Final R-peak location update
ecg_locs = []  # Initialize the ecg_locs vector
ecg_locs.extend(temp_locs)

In [ ]:
# ecg_locs

In [ ]:
temp_locs

In [ ]:
# Print the final R-peak locations
# print("Final R-peak locations (ecg_locs):", ecg_locs)

In [ ]:
import matplotlib.pyplot as plt

# Plotting the beat_detected_signal
plt.figure(figsize=(10, 6))
plt.plot(beat_detected_signal[0], color='blue', label='Beat Detected Signal')

# Plotting the R-peaks
# plt.scatter(missed_peaks, beat_detected_signal.flatten()[missed_peaks], color='red')
plt.plot(missed_peaks, beat_detected_signal[0][missed_peaks], 'gx', markersize=2,label='missed R-Peaks')
plt.plot(peaks, beat_detected_signal[0][peaks], 'ro', markersize=2,label='intial R-Peaks')

# Labeling the axes and adding a legend
plt.xlabel('Sample')
plt.ylabel('Amplitude')
plt.legend()

# Display the plot
plt.show()


In [ ]:
# Load CSV file into a DataFrame
df = pd.read_csv('/content/drive/MyDrive/ECG/incart_DB/Incart_csv/I01.csv')

# Extract Lead II data
lead_ii_data = df['II'].values

# Apply peak detection algorithm

peaks_pos, _ = find_peaks(beat_detected_signal.flatten(), height=pk_thr, distance=rr_thr)
peaks_neg, _ = find_peaks(-beat_detected_signal.flatten(), height=pk_thr, distance=rr_thr)

# Combine positive and negative peaks
peaks = np.concatenate((peaks_pos, peaks_neg))

# Count the total number of beats
total_beats = len(beat_detected_signal)

# Print the result
print("Total beats:", total_beats)


In [ ]:
peaks

In [ ]:
# ecg_locs

In [ ]:
beat_detected_signal.shape

In [ ]:
import numpy as np
from scipy.signal import find_peaks

# Flatten the beat_detected_signal to have shape (180, 3600)
beat_detected_flat = np.squeeze(beat_detected_signal)

# Calculate the sum of all data points in beat_detected_signal
total_signal_amplitude = np.sum(np.abs(beat_detected_flat))

# Initialize total positive peak value
total_positive_peak_value = 0

# Iterate over samples
for sample in beat_detected_flat:
    # Find indices of positive peaks
    positive_peaks_indices, _ = find_peaks(sample, height=0)

    # Get positive peak values
    positive_peak_values = sample[positive_peaks_indices]

    # Sum positive peak values
    total_positive_peak_value += np.sum(np.abs(positive_peak_values))

# Calculate the percentage of positive peak value
percentage_positive_peak_value = (total_positive_peak_value / total_signal_amplitude) * 100

# Print the result
print("Total positive peak value percentage:", percentage_positive_peak_value)


In [ ]:
total_signal_amplitude

In [ ]:
positive_peaks_indices

In [ ]:
positive_peak_values

In [ ]:
total_positive_peak_value

In [ ]:
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [ ]:
y_actual = total_beats
y_predicted = beat_detected_signal[0]

tn, fp, fn, tp = confusion_matrix(y_actual, y_predicted).ravel()
f1_score = (2tp)/((2tp)+fp+fn)
ppv = tp/(tp+fp)
sens = tp/(tp+fn)
confusion_matrix = metrics.confusion_matrix(y_actual, y_predicted)
cm_display = metrics.ConfusionMatrixDisplay(confusion_matrix = confusion_matrix, display_labels = [False, True])
cm_display.plot()
plt.show()

In [ ]:

# positive_peak_value = max(positive_peak_values.flatten())
# print("Positive peak value:", positive_peak_value)


In [ ]:
import numpy as np
import pandas as pd
from scipy.signal import find_peaks

# Load the signal data from the CSV file
data = pd.read_csv('/content/drive/MyDrive/ECG/incart_DB/Incart_csv/I01.csv')

# Extract the relevant signal column from the DataFrame
signal = data['II'].values

# Apply peak detection to identify positive peaks
positive_peaks, _ = find_peaks(signal, height=0)

# Store the positions or indices of positive peaks as reference annotations
reference_annotations = positive_peaks

# Save reference annotations to a file or use as needed
np.savetxt('reference_annotations.csv', reference_annotations, delimiter=',')


In [ ]:
ann = pd.read_csv("/content/reference_annotations.csv")

In [ ]:
ann

In [ ]:
import numpy as np

# Flatten the beat_detected_signal to have shape (180, 3600)
beat_detected_flat = np.squeeze(beat_detected_signal)

# Initialize true positive (TP) and false negative (FN) counts
TP = 0
FN = 0

# Load reference annotations from 'reference_annotations.csv'
reference_annotations = np.loadtxt('reference_annotations.csv', delimiter=',')

# Iterate over samples
for i in range(len(beat_detected_flat)):
    detected_peaks = beat_detected_flat[i]

    # Compare each detected peak with reference annotations
    for peak in detected_peaks:
        # Check if the peak exists in the reference annotations for the corresponding sample
        if np.any(np.isclose(peak, reference_annotations[i], atol=1e-5)):
            TP += 1
        else:
            FN += 1

# Calculate sensitivity
sensitivity = FN / (TP + FN)

# Print the result
print("Sensitivity:", sensitivity)


In [ ]:
import numpy as np

# Flatten the beat_detected_signal to have shape (180, 3600)
beat_detected_flat = np.squeeze(beat_detected_signal)

# Load reference annotations from 'reference_annotations.csv'
reference_annotations = np.loadtxt('reference_annotations.csv', delimiter=',')

# Initialize variables
total_error = 0
num_samples = len(beat_detected_flat)

# Iterate over samples
for i in range(num_samples):
    detected_peaks = beat_detected_flat[i]

    # Compare each detected peak with reference annotations
    for peak in detected_peaks:
        # Find the closest reference annotation to the detected peak
        closest_annotation = np.min(np.abs(reference_annotations[i] - peak))

        # Accumulate the error
        total_error += closest_annotation

# Calculate the mean error in terms of samples
mean_error_samples = total_error / (num_samples * len(beat_detected_flat[0]))

# Convert mean error to milliseconds using the sampling rate (360 samples per second)
sampling_rate = 360
mean_error_ms = mean_error_samples * (1000 / sampling_rate)

# Print the result
print("Mean error:", "%.2f" % mean_error_ms, "ms")
print("(~" + "2" + " samples)")


In [ ]:
beat_detected_flat

# **Rough-I**

In [ ]:
# Initialization and Definitions
Fs = 360  # Sampling frequency (Hz)
pk_thr = 0.25  # R-peak threshold
rr_thr = int(0.25 * fs)  # RR-interval threshold
missed_thr = int(0.15 * fs)  # Minimum separation threshold for missed beats
temp_locs = []  # Temporary peak locations
pk_vec = []  # Peak amplitudes
ppi_vec = []  # Peak-to-peak intervals
ecg_locs = []  # Actual R-peak locations


In [ ]:
from scipy.signal import find_peaks

# Define the threshold for initial peak detection
pk_thr = 0.25  # R-peak threshold
rr_thr = int(0.25 * Fs)  # RR-interval threshold (minimum RR-interval between consecutive R-peaks)

# Perform initial peak detection on each reconstructed segment
for segment_index, segment in enumerate(beat_detected_signal):
    # Extract the denoised ECG signal
    ecg_signal = segment[:, 0]

    # Find the positive peaks (R-peaks) in the ECG signal using the provided thresholds
    positive_peaks, _ = find_peaks(ecg_signal, height=pk_thr, distance=rr_thr)

    # Invert the ECG signal
    inverted_ecg_signal = -ecg_signal

    # Find the negative peaks in the inverted ECG signal using the provided thresholds
    negative_peaks, _ = find_peaks(inverted_ecg_signal, height=pk_thr, distance=rr_thr)

    # Plot the segment with detected peaks
    plt.figure(figsize=(10, 4))
    plt.plot(ecg_signal)
    plt.plot(positive_peaks, ecg_signal[positive_peaks], 'ro', markersize=4, label='Positive Peaks')
    plt.plot(negative_peaks, ecg_signal[negative_peaks], 'bo', markersize=4, label='Negative Peaks')
    plt.xlabel('Time')
    plt.ylabel('Denoised ECG Signal')
    plt.title(f'Reconstructed Segment {segment_index+1} with Initial Peak Detection')
    plt.legend()
    plt.show()


In [ ]:
positive_peaks

In [ ]:
negative_peaks

In [ ]:
segment_length1 = 10 * 360  # 10 seconds of ECG data at 360 Hz sampling frequency
# num_segments1 = len(ecg_signal) // segment_length1

In [ ]:
num_segments1 = len(segments)

In [ ]:
num_segments1

In [ ]:
peaks

In [ ]:
ecg_locs

# **Rough-II**

In [ ]:
# /content/drive/MyDrive/ECG/incart_DB/Incart_csv/I01.csv

In [ ]:
pip install peakutils

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import find_peaks

# Step 1: Load the ECG data from CSV file
# data = pd.read_csv('/content/drive/MyDrive/ECG/incart_DB/Incart_csv/I01.csv')

# Step 2: Extract the ECG signal
# ecg_signal = beat_detected_signal[0]

# Step 3: Preprocessing - Apply bandpass filter, differentiation, and squaring
# You can implement these preprocessing steps based on the characteristics of your data and requirements.

# Step 4: Perform R-peak detection considering both polarities
positive_peaks, _ = find_peaks(ecg_signal, distance=100, height=0.5, prominence=0.3)
negative_peaks, _ = find_peaks(-ecg_signal, distance=100, height=0.5, prominence=0.3)
r_peaks = np.concatenate((positive_peaks, negative_peaks))

# Step 5: Plot the ECG signal with detected R-peaks
plt.figure(figsize=(12, 6))
plt.plot(ecg_signal, color='blue', label='ECG Signal')
plt.scatter(r_peaks, ecg_signal[r_peaks], color='red', label='R-Peaks')
plt.xlabel('Sample')
plt.ylabel('ECG Amplitude')
plt.title('ECG Signal with Detected R-Peaks')
plt.legend()
plt.show()


In [ ]:
filtered_ecg_signal

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, UpSampling1D

# Step 1: Prepare the ECG data
data = pd.read_csv('/content/drive/MyDrive/ECG/incart_DB/Incart_csv/I01.csv')
ecg_signal = data['II'].values

# Step 2: Split the data into training and testing sets
train_data, test_data = train_test_split(ecg_signal, test_size=0.2, random_state=42)

# Step 3: Build the autoencoder model
model = Sequential()
model.add(Conv1D(filters=32, kernel_size=5, activation='relu', padding='same', input_shape=(train_data.shape[0], 1)))
model.add(MaxPooling1D(pool_size=2, padding='same'))
model.add(Conv1D(filters=16, kernel_size=3, activation='relu', padding='same'))
model.add(MaxPooling1D(pool_size=2, padding='same'))
model.add(Conv1D(filters=8, kernel_size=3, activation='relu', padding='same'))
model.add(MaxPooling1D(pool_size=2, padding='same'))
model.add(Conv1D(filters=8, kernel_size=3, activation='relu', padding='same'))
model.add(MaxPooling1D(pool_size=2, padding='same'))
model.add(UpSampling1D(size=2))
model.add(Conv1D(filters=8, kernel_size=3, activation='relu', padding='same'))
model.add(UpSampling1D(size=2))
model.add(Conv1D(filters=16, kernel_size=3, activation='relu', padding='same'))
model.add(UpSampling1D(size=2))
model.add(Conv1D(filters=32, kernel_size=5, activation='relu', padding='same'))
model.add(UpSampling1D(size=2))
model.add(Conv1D(filters=1, kernel_size=5, activation='relu', padding='same'))

# Step 4: Compile and train the autoencoder
model.compile(optimizer='adam', loss='mse')
train_data = train_data.reshape(train_data.shape[0], 1)
test_data = test_data.reshape(test_data.shape[0], 1)
model.fit(train_data, train_data, epochs=10, batch_size=32, validation_data=(test_data, test_data))


# Step 5: Extract potential QRS complex locations
decoded_data = model.predict(test_data)
# potential_qrs_locations = np.argmax(decoded_data, axis=1)

# Step 6: Plot the original and reconstructed signals
plt.figure(figsize=(12, 6))
plt.plot(test_data[0], color='blue', label='Original')
plt.plot(decoded_data[0], color='red', label='Reconstructed')
plt.scatter(decoded_data[0], color='green', label='Potential QRS')
plt.xlabel('Sample')
plt.ylabel('ECG Amplitude')
plt.title('Original and Reconstructed Signals with Potential QRS Complex Locations')
plt.legend()
plt.show()


In [ ]:
train_data.shape

In [ ]:
test_data.shape

In [ ]:
decoded_data.shape[0]

In [ ]:
plt.plot(decoded_data[1][1])

In [ ]:
# Step 6: Plot the original and reconstructed signals
plt.figure(figsize=(12, 6))
plt.plot(test_data[0], color='blue', label='Original')
plt.plot(decoded_data[0], color='red', label='Reconstructed')
plt.scatter(decoded_data[0], color='green', label='Potential QRS')
plt.xlabel('Sample')
plt.ylabel('ECG Amplitude')
plt.title('Original and Reconstructed Signals with Potential QRS Complex Locations')
plt.legend()
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def detect_peaks(data, threshold):
    peaks = []
    for i in range(1, len(data) - 1):
        if data[i] > threshold and data[i] > data[i-1] and data[i] > data[i+1]:
            peaks.append(i)
        elif data[i] < -threshold and data[i] < data[i-1] and data[i] < data[i+1]:
            peaks.append(i)
    return peaks

# Example data
data = [1, -2, 3, -4, 5, -6, 7, -8, 9]
threshold = 0

# Detect positive and negative peaks
positive_peaks = detect_peaks(data, threshold)
negative_peaks = detect_peaks([-x for x in data], threshold)

# Plot the data
plt.plot(data, label="Data")

# Highlight positive peaks
plt.scatter(positive_peaks, [data[i] for i in positive_peaks], color='r', label="Positive Peaks")

# Highlight negative peaks
plt.scatter(negative_peaks, [-data[i] for i in negative_peaks], color='g', label="Negative Peaks")

plt.xlabel("Index")
plt.ylabel("Value")
plt.legend()
plt.show()
